# Elastic workflow

This is a elastic workflow that is based on a stress-strain approach.

If you want to read more about this approach, please check out [https://docs.materialsproject.org/methodology/materials-methodology/elasticity](https://docs.materialsproject.org/methodology/materials-methodology/elasticity) and the following publication [https://www.nature.com/articles/sdata20159](https://www.nature.com/articles/sdata20159).


## Let's start with the workflow

We now simply load the force field elastic workflow. We change from the default universal ML potential to M3GNet. For CHGNet, an updated version of the ML potential needs to be used: [https://github.com/CederGroupHub/chgnet/issues/79](https://github.com/CederGroupHub/chgnet/issues/79)



In [ ]:
from jobflow import run_locally
from pymatgen.core import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

from atomate2.forcefields.flows.elastic import ElasticMaker
from atomate2.forcefields.jobs import M3GNetRelaxMaker

structure = Structure(
    lattice=[[0, 2.73, 2.73], [2.73, 0, 2.73], [2.73, 2.73, 0]],
    species=["Si", "Si"],
    coords=[[0, 0, 0], [0.25, 0.25, 0.25]],
)

structure= SpacegroupAnalyzer(structure).get_primitive_standard_structure()

elastic_flow = ElasticMaker(
    bulk_relax_maker=M3GNetRelaxMaker(
        relax_cell=True, relax_kwargs={"fmax": 0.00001}
    ),
    elastic_relax_maker=M3GNetRelaxMaker(
        relax_cell=False, relax_kwargs={"fmax": 0.00001}
    ),
).make(structure)


## Let's have a look at the computing jobs
The phonon run will first perform a bulk relaxation, then the displacements are generated and run.
As we currently don't have a way to compute BORN charges with such potentials, a non-analytical term correction is not performed here.


In [ ]:
elastic_flow.draw_graph().show()

## Let's execute the workflow

In [ ]:
%%capture
from jobflow import run_locally

responses = run_locally(elastic_flow,  create_folders=True)

## Let's have a look at the outputs

We query our database for the relevant outputs (here results from the elastic document)


In [25]:
from jobflow import SETTINGS

store = SETTINGS.JOB_STORE
store.connect()

result = store.query_one(
    {"name": "fit_elastic_tensor"},
    properties=[
        "output.elastic_tensor",
        "output.derived_properties",
    ],
    load=True,
    sort={"completed_at": -1} # to get the latest computation
)

You can then access the elastic tensor in different output formats:

In [ ]:
print(result["output"]["elastic_tensor"]["ieee_format"])


And, derived properties such as bulk moduli can be accessed:

In [ ]:
print(result["output"]["derived_properties"])